In [6]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [7]:
import mlflow

# set tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

#create a new MLflow Experiment
mlflow.set_experiment("MLflow for Fraud detection")

2024/06/25 00:06:45 INFO mlflow.tracking.fluent: Experiment with name 'MLflow for Fraud detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/668097690514392710', creation_time=1719263205613, experiment_id='668097690514392710', last_update_time=1719263205613, lifecycle_stage='active', name='MLflow for Fraud detection', tags={}>

In [10]:
# Load the datasets
fraud_data = pd.read_csv('../data/old/Fraud_Data.csv')
creditcard_data = pd.read_csv('../data/old/creditcard.csv')
ip_data = pd.read_csv('../data/old/IpAddress_to_Country.csv')

fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

fraud_data['signup_hour'] = fraud_data['signup_time'].apply(lambda x: x.hour)
fraud_data['signup_day'] = fraud_data['signup_time'].apply(lambda x: x.day)
fraud_data['purchase_hour'] = fraud_data['purchase_time'].apply(lambda x: x.hour)
fraud_data['purchase_day'] = fraud_data['purchase_time'].apply(lambda x: x.day)

fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])


In [11]:
fraud_data.head(5)

,user_id,purchase_value,device_id,source,browser,sex,age,ip_address,class,signup_hour,signup_day,purchase_hour,purchase_day
0,22058,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,22,24,2,18
1,333320,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,20,7,1,8
2,1359,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,1,18,1
3,150084,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,21,28,13,4
4,221365,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,7,21,18,9
